# Explanation of my solution

In this notebook, I am going through the solution code in its order of execution as well as through the thinking process I had to find this solution to the problem.

I will name this solution: **Moving the chairs to the room names**.

You can follow this code along in the `script.py` file at the root of the directory.



In [2]:


from utils_notebook import get_project_root, notebook_print_of

ROOT = get_project_root()
EXAMPLES_FOLDER = str(ROOT) + '/explanatory_rooms'

## Overall organization of the code

In the `script.py` file we have as:
- **Input**: a path to the `txt` file reprensenting an apartment. *ex: rooms.txt*
- **Output**: a print in the console of the excepted result, namely the:
```
- Number of different chair types for the apartment
- Number of different chair types per room
```
as mentioned in the **task_en.txt** file.

To perform this task, it runs the `run_solution()` function

The `run_solution()` is divided in 4 steps:

- **Step 1:** Read the `txt` file
- **Step 2:** Locate the chairs in the apartment
- **Step 3:** Find the room of each chair
- **Step 4:** Transform the result and display it

## Preliminary steps

### Step 1: Read and transform the txt file

I simply read the `room.txt` file or any other `txt` file of the same type and tranform it into a list of lists that I name `apartment_init`. 

I chose this *list of lists* data structure, i.e. to easily track any point in the apartment with coordinates: i, j as follows: `apartment_init[i][j]` which will be used in the rest of the code.

For debugging purposes, note that an apartment as a list of lists can be displayed in the console or the debugger with the function `console_print_of()` present in the `utils.py` file.

### Step 2: Locate chairs in the apartment

1. I scan throught every single pair of coordinates `(i, j)` to build the dictionary: `dict_pos_chairs` with:
    - **key**: (i, j) the coordinates of the chairs
    - **values**: the coresponding letter of chair in the apartment

2. In addition to this dictionary, I also create a list `list_pos_chairs` which gives the positions of chairs ordered vertically and horizontally. Sorting the list this way allows to easily find which chair is treated when we loop over each of them.



## Step 3: Find the room of each chair 
### (Main step)

This step is divided into 3 sub-steps:
- **Step 3.1:** Temporarily remove the other chairs in the apartment
- **Step 3.2:** Search the room of a chair
- **Step 3.3:** Save the result

This hole step is performed looping over the position of the chairs: `list_pos_chairs`. Within the loop the variables are:
- `k`: the index in `list_pos_chairs` of the chair. Having the `txt` file open, as `list_pos_chairs` is sorted it is fairly easy to locate the chair with `k`.
- `coord = (i, j)` : the coordinates of the chair in the `apartment`
-  `chair = apartment_init[i][j]`: the letter of the chair


### Step 3.1: Temporarily remove the other chairs in the apartment

We are now inside the **for** loop and therefore the variables above-mentioned will be used in the explanation.

As explained in the introduction, my method consists in moving the letter of each chair. To avoid any kind of conflicts between two chairs letters, I decided to create a copy (or rather a [deepcopy](https://docs.python.org/3/library/copy.html)) of `apartment_init` called `apartment` in which the other chair letters than `chair` will be removed.

This is how `apartment` looks like when treating the chair `'S'` located in `i, j = (4, 10)`

In [3]:
notebook_print_of(f'{EXAMPLES_FOLDER}/1.rooms_one_chair.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                            S       |',
 '|           |         (sleeping room)            |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           +--------------+---------------------+',
 '|                          |                     |',
 '|                          |                     |',
 '|                          |    (office)         |',
 '|                          |                     |',
 '+--------------+           |                     |',
 '|              |           |                     |',
 '| (toile

### Step 3.2: Search the room of a chair

#### (The core of the algorithm)

For this step, we call the function `search_room()` that has as inputs:
- `apartment`: the apartment with only on chair present
- `coord`: the coordinates of the chair
an return:
- `room_of_chair`: the room name, the chair belongs to

In the above example:
```
search_room(apartment, (4, 10)) = 'sleeping room'
```

The best way to explain the logic behind this piece of code is I think to go through the different ideas that I had.

#### First idea: room name on the same row

The idea consists of:

1. Checking if a chair located in `(i, j)` is in the same row as its room name.
2. Saving this room name

Building the object `apartment` as a list of lists, it fairly easy to extract a given row `i`. We just need to define: `row = apartment[i]`

If we consider this situation:

In [4]:
notebook_print_of(f'{EXAMPLES_FOLDER}/2.chair_room_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|  (closet) |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |         (sleeping room)     S      |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

In this situation, asserting if the chair has its room name on the same row can be done by checking if a paranthesis belongs to this row.

However, if we are in this situation:

In [5]:
notebook_print_of(f'{EXAMPLES_FOLDER}/3.not_good_room_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '| (closet)  |              S                     |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

Visually, it is obvious to say that the chair `S` is in the room `sleeping room`. But the row where `S` is contains another room name: `closet`.

The strategy of finding a paranthesis on the same row, will give a false result in this case.

The right approach is:
- to check if a room name is on the same row as a chair
- if so, check if exclusively `' '` characters are between the chair letter and the room name on the same row.

This is performed by the function `is_room_on_same_row()`, with an equivalent method:
- we remove `' '` from the row
- we check if directly preceded by a `)` or followed by a `(` as room names are wrapped in them.

As the second step of this first idea, if `is_room_on_same_row(apartment, i, j)` has a `True` value, we want to save the found room name. 

This task is performed by the function `found_room_on_same_row()` in this way, we go through the `row = apartment[i]`:

1. We start from the chair in `row[j]`
2. We create two indices `j_left` and `j_right`
3. We move `j_left` on the left and `j_right` on the right, while no paranthesis is found. It obviously terminates as `is_room_on_same_row(apartment, i, j)` is `True`. 

***Note:*** *Notice that the value of `j_left` and `j_right` is only updated when not encountering an element of `sep_chars`. This ensures to only stay within the room*. `sep_chars` are the characters of separation of a room that you can find in the `special_characters.py` file.

4. Once a paranthesis is encountered, we set the string `side` which is in `{'left', 'right'}` depending on where the paranthesis was found.
5. We finally build the room name character after character on either side depending on the value of `side`.

#### Second idea: Moving vertically and explore horizontally

So far, I have been able to find a room name of a chair if they were both on the same line. But what if that was not the case as in the example below?

In [6]:
notebook_print_of(f'{EXAMPLES_FOLDER}/4.room_not_on_same_row.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                                    |',
 '|           |      S                             |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            |                                    |',
 '            +------------------------------------+']

The idea I got, was to vertically displace the chair so that it lands on a row where its room name is.

This deplacement is performed by the function `change_pos()` which has as inputs:
- `apartment`
- `i`, `j`: the old coordinates
- `new_i`, `new_j`: the coordinates of the spot we want to move the chair to

The function:
1. switches the characters at `(i, j)` and `(new_i, new_j)`
2. returns the updated coordinates.

Now that we know how to move a chair around, let's see how we can move vertically to find the room name.

I decided to:

1. Move the chair to the empty space just above it until we reach the top of the room, i.e., the above character is in `sep_chars`.

2. If the top of the room is reached, get back to the initial position.

3. Move the chair to the empty space just below it until we reach the bottom of the room, i.e., the below character is in `sep_chars`.

At each step of this procedure, we check if the room name is on the same row, we break it.


This task is perfomed the `explore_vertical_moves()` function. This function does other things on the side that we will explain later but the core of the function is explained above.

Here are a few examples. The numbers give the positions in order of the different positions that the chair `S` takes. 


In [7]:
notebook_print_of(f'{EXAMPLES_FOLDER}/5.horizontal_exploration_1.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |    (sleeping room)             3   |',
 '|           |                                2   |',
 '+-----------+                                1   |',
 '            |                                S   |',
 '            |                                    |',
 '            +------------------------------------+']

In [8]:
notebook_print_of(f'{EXAMPLES_FOLDER}/6.horizontal_exploration_2.txt')

['+-----------+------------------------------------+',
 '|           |      2                             |',
 '| (closet)  |      1                             |',
 '|           |      S                             |',
 '+-----------+      3                             |',
 '            |      4       (sleeping room)       |',
 '            |                                    |',
 '            +------------------------------------+']

#### ALERT: Problem if the chair is under the room

Moving vertically and exploring horizontally seems to work fine, if of course we can move vertically. What if one of the character of the room name or one of the paranthesis that wraps it is on the same **column** as `chair`. If we try to move the chair up and down, it will eventually "bump into" the room name which will create a mess.

Here is an example of such a case:

In [11]:
notebook_print_of(f'{EXAMPLES_FOLDER}/7.chair_and_room_on_same column.txt')


['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                 S                  |',
 '|           |                                    |',
 '+-----------+                                    |',
 '            |              (sleeping room)       |',
 '            |                                    |',
 '            +------------------------------------+']

The solution I found to this problem is to introduce a **vertical check** before moving the chair up and down. This is performed by the `is_room_on_same_column()`

The approach is similar to that of `is_room_on_same_column()` but once the `' '` are removed, instead of looking for a paranthesis, we look for a character that is not in `sep_char`, i.e., a letter or a paranthesis.

In the example above, the column of `S` when the `' '` are removed, is:
`['-', 'S', 'e', '-']` as `'e'` follows `'S'` and is not in `sep_chars`, there is a room name on the same column.

***Note:*** *There is a side case to this problem of this problem. I will tackle it in the next notebook*

When the `is_room_on_same_column()` is `True`, there is no need to move the chair up and down. Instead we use the function `find_room_on_same_column()`.

This function, as its equivalent `find_room_on_same_row()`:

1. We start from the chair in `col[i]`
2. We create two indices `i_up` and `j_down`
3. We move `i_up` up and `i_dowm` down, while a paranthesis is not found. It obviously terminates as `is_room_on_same_row(apartment, i, j)` is `True`. 

***Note:*** *Notice that the value of `j_left` and `j_right` is only updated when not encountering an element of `sep_chars`. This ensures to only stay within the room*. `sep_chars` are the characters of separation of a room that you can find in the `special_characters.py` file.

4. Once a paranthesis is encountered, we set the string `side` which is in `{'left', 'right'}` depending on where the paranthesis was found.
5. We finally build the room name character after character on either side depending on the value of `side`.